In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# 1. Data Preparation (same as before)
df = pd.read_csv('DailyDelhiClimate.csv')
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)

features = ['meantemp', 'humidity', 'wind_speed', 'meanpressure']
df = df[features]

scalers = {}
for column in df.columns:
    scaler = MinMaxScaler()
    df[column] = scaler.fit_transform(df[[column]])
    scalers['scaler_' + column] = scaler

n_past = 10
n_future = 5
n_features = len(features)

X = []
y = []
for i in range(n_past, len(df) - n_future + 1):
    X.append(df.iloc[i - n_past:i].values)
    y.append(df.iloc[i:i + n_future].values)

X = np.array(X)
y = np.array(y)

train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# 2. LSTM Encoder-Decoder Models

# E1D1 Model with LSTM
encoder_inputs_e1d1 = tf.keras.layers.Input(shape=(n_past, n_features))
encoder_lstm_e1d1 = tf.keras.layers.LSTM(16, return_state=True)
encoder_outputs_e1d1, state_h_e1d1, state_c_e1d1 = encoder_lstm_e1d1(encoder_inputs_e1d1)
encoder_states_e1d1 = [state_h_e1d1, state_c_e1d1]

decoder_inputs_e1d1 = tf.keras.layers.RepeatVector(n_future)(encoder_outputs_e1d1)
decoder_lstm_e1d1 = tf.keras.layers.LSTM(16, return_sequences=True)
decoder_outputs_e1d1 = decoder_lstm_e1d1(decoder_inputs_e1d1, initial_state=encoder_states_e1d1)
decoder_dense_e1d1 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(n_features))
decoder_outputs_e1d1 = decoder_dense_e1d1(decoder_outputs_e1d1)

model_e1d1_lstm = tf.keras.models.Model(encoder_inputs_e1d1, decoder_outputs_e1d1)


# E2D2 Model with LSTM (2 layers each)
encoder_inputs_e2d2 = tf.keras.layers.Input(shape=(n_past, n_features))

encoder_lstm1_e2d2 = tf.keras.layers.LSTM(16, return_sequences=True, return_state=True)
encoder_outputs1_e2d2, state_h1_e2d2, state_c1_e2d2 = encoder_lstm1_e2d2(encoder_inputs_e2d2)
encoder_states1_e2d2 = [state_h1_e2d2, state_c1_e2d2]

encoder_lstm2_e2d2 = tf.keras.layers.LSTM(16, return_state=True)
encoder_outputs2_e2d2, state_h2_e2d2, state_c2_e2d2 = encoder_lstm2_e2d2(encoder_outputs1_e2d2)
encoder_states2_e2d2 = [state_h2_e2d2, state_c2_e2d2]


decoder_inputs_e2d2 = tf.keras.layers.RepeatVector(n_future)(encoder_outputs2_e2d2)

decoder_lstm1_e2d2 = tf.keras.layers.LSTM(16, return_sequences=True)
decoder_outputs1_e2d2 = decoder_lstm1_e2d2(decoder_inputs_e2d2, initial_state=encoder_states1_e2d2)

decoder_lstm2_e2d2 = tf.keras.layers.LSTM(16, return_sequences=True)
decoder_outputs2_e2d2 = decoder_lstm2_e2d2(decoder_outputs1_e2d2, initial_state=encoder_states2_e2d2)

decoder_dense_e2d2 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(n_features))
decoder_outputs_e2d2 = decoder_dense_e2d2(decoder_outputs2_e2d2)

model_e2d2_lstm = tf.keras.models.Model(encoder_inputs_e2d2, decoder_outputs_e2d2)


# 3. Compilation and Training (same as before)
reduce_lr = tf.keras.callbacks.LearningRateScheduler(lambda x: 1e-3 * 0.90 ** x)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

model_e1d1_lstm.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.Huber())
history_e1d1_lstm = model_e1d1_lstm.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), batch_size=16, verbose=1, callbacks=[reduce_lr, early_stopping])

model_e2d2_lstm.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.Huber())
history_e2d2_lstm = model_e2d2_lstm.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), batch_size=16, verbose=1, callbacks=[reduce_lr, early_stopping])

# 4. Prediction and Inverse Scaling (same as before, but use the correct model names)
pred_e1d1_lstm = model_e1d1_lstm.predict(X_test)
pred_e2d2_lstm = model_e2d2_lstm.predict(X_test)

for index, i in enumerate(features):
    scaler = scalers['scaler_' + i]
    pred_e1d1_lstm[:, :, index] = scaler.inverse_transform(pred_e1d1_lstm[:, :, index])
    pred_e2d2_lstm[:, :, index] = scaler.inverse_transform(pred_e2d2_lstm[:, :, index])
    y_train[:, :, index] = scaler.inverse_transform(y_train[:, :, index])
    y_test[:, :, index] = scaler.inverse_transform(y_test[:, :, index])

# 5. Evaluation (same as before, but use the correct prediction variables)
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

y_true_all = y_test.flatten()

y_pred_e1d1_all = pred_e1d1_lstm.flatten() # Use predictions from the LSTM models
y_pred_e2d2_all = pred_e2d2_lstm.flatten()

# ... (rest of the evaluation code remains the same, just use the _lstm variables)

mae_e1d1 = mean_absolute_error(y_true_all, y_pred_e1d1_all)
mse_e1d1 = mean_squared_error(y_true_all, y_pred_e1d1_all)
rmse_e1d1 = np.sqrt(mse_e1d1)
mape_e1d1 = mean_absolute_percentage_error(y_true_all, y_pred_e1d1_all)
r2_e1d1 = r2_score(y_true_all, y_pred_e1d1_all)

# Define/calculate mse_e2d2 here:
mse_e2d2 = mean_squared_error(y_true_all, y_pred_e2d2_all)
rmse_e2d2 = np.sqrt(mse_e2d2)
mape_e2d2 = mean_absolute_percentage_error(y_true_all, y_pred_e2d2_all)
r2_e2d2 = r2_score(y_true_all, y_pred_e2d2_all)

# Display results in a structured table
import pandas as pd
results_df = pd.DataFrame({
    "Model": ["E1D1", "E2D2"],
    "MAE": [mae_e1d1, mae_e2d2],
    "MSE": [mse_e1d1, mse_e2d2],
    "RMSE": [rmse_e1d1, rmse_e2d2],
    "MAPE": [mape_e1d1, mape_e2d2],
    "R2 Score": [r2_e1d1, r2_e2d2]
})

print(results_df)

from sklearn.metrics import mean_squared_error
import numpy as np

# Reshape y_test and pred_e1d1 to 2D
y_test_2d = y_test.reshape(-1, y_test.shape[-1])  # Reshape to (samples * time steps, features)

# Get prediction from the bilstm model
# Instead of pred_e1d1, use pred_e1d1_lstm (output from your E1D1 LSTM model)
pred_e1d1_lstm_2d = pred_e1d1_lstm.reshape(-1, pred_e1d1_lstm.shape[-1])  # Reshape using pred_e1d1_lstm

# Normalize MSE using variance of actual data
y_var = np.var(y_test_2d)  # Variance of true values
# Use pred_e1d1_lstm_2d instead of pred_e1d1_2d
normalized_mse_e1d1 = mean_squared_error(y_test_2d, pred_e1d1_lstm_2d) / y_var

# ... (similarly, use pred_e2d2_lstm for E2D2 calculations) ...

pred_e2d2_lstm_2d = pred_e2d2_lstm.reshape(-1, pred_e2d2_lstm.shape[-1])  # Reshape using pred_e2d2_lstm
# Use pred_e2d2_lstm_2d instead of pred_e2d2_2d
normalized_mse_e2d2 = mean_squared_error(y_test_2d, pred_e2d2_lstm_2d) / y_var


print("Normalized MSE E1D1:", normalized_mse_e1d1)
print("Normalized MSE E2D2:", normalized_mse_e2d2)